In [1]:
import os,sys
import time,datetime
import pickle
import numpy as np
import pandas as pd
import time
groundtruth = '../dataset/open_peer_review_v3/peer_review/translated_groundtruth.csv'
peer_review = '../dataset/open_peer_review_v3/peer_review/peer_review_forPG3.csv'
gDF = pd.read_csv(groundtruth)
rDF = pd.read_csv(peer_review)

In [12]:
%run ../tools/PG5PC2a.py
modelPG5PC2a = PG5PC2a(gDF,rDF)

### テスト: corrcoefWithTruth

In [13]:
start = time.time()
car = modelPG5PC2a.corrcoefWithTruth([0.6094, 3.8560, 11.2887, 10.9740, 66.0071, 12.9332, 1.0])
elapsed_time = time.time() - start
print('{} sec'.format(elapsed_time))

213.123023033 sec


In [14]:
car

0.45607032270438408

In [5]:
f = open('../result/inferred_parameter/PG1/PG1-20171208162003.pkl','rb')
est = pickle.load(f)
np.corrcoef(modelPG1.grade, est['ability'])[0,1]

0.49079361863447923

### メイキング: 推定パラメータの保存と、モデル表作成

In [4]:
start = time.time()
pg1_fit = modelPG1.fit([2.24705059, 2.64350654, 62.67778188, 78.70005719, 1.33529373])
elapsed_time = time.time() - start
print('{} sec'.format(elapsed_time))

112.192134142 sec


#### パラメータ抽出方法の確認

古いパラメータ抽出方法

In [5]:
pg1_eap = pg1_fit.summary()['summary'][:,0]
pg1_estimated = pg1_eap[0:modelPG1.userNum]
np.corrcoef(modelPG1.grade, pg1_estimated)[0,1]

0.49065246361026571

新しいパラメータ抽出方法

In [6]:
ext = pg1_fit.extract()
type(ext), ext.keys()

(collections.OrderedDict, [u'ability', u'reliability', u'bias', u'lp__'])

In [15]:
estimated = ext['ability'].mean(axis=0)
np.corrcoef(modelPG1.grade, estimated)[0,1]

0.49065246361026771

In [53]:
inferred_reliability = ext['reliability'].mean(axis=0)
inferred_bias = ext['bias'].mean(axis=0)

#### パラメータ保存

In [54]:
save_param_dir = '../result/inferred_parameter/PG1'
def saveStanExtract(var, pkl_file):
    with open(pkl_file,'wb') as f:
        pickle.dump(var,f)
def timeStamp():
    # return time as str
    todaydetail = datetime.datetime.today()
    return todaydetail.strftime("%Y%m%d%H%M%S")

In [55]:
ext = pg1_fit.extract()
inferred_ability = ext['ability'].mean(axis=0)
inferred_reliability = ext['reliability'].mean(axis=0)
inferred_bias = ext['bias'].mean(axis=0)
parameters = {'ability':inferred_ability, 'reliability':inferred_reliability, 'bias':inferred_bias}
pkl_name = 'PG1-{}.pkl'.format(timeStamp())
saveStanExtract(parameters, os.path.join(save_param_dir,pkl_name))

読み込み

In [57]:
f = open(os.path.join(save_param_dir,pkl_name),'rb')
est = pickle.load(f)
np.corrcoef(modelPG1.grade, est['ability'])[0,1]

0.49065246361026771

#### モデル表の作成

In [58]:
save_param_dir = '../result/inferred_parameter/PG1'
def recordPG1Info(hyper_list, model_file, csv_file):
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
    else:
        df = pd.DataFrame(columns=['file','mu0','gamma0','alpha0','beta0','eta0'])
    add_srs = pd.Series([model_file]+hyper_list, index=df.columns)
    df = df.append(add_srs, ignore_index=True)
    df.to_csv(csv_file, index=False)

In [59]:
recordPG1Info([2.24705059, 2.64350654, 62.67778188, 78.70005719, 1.33529373],pkl_name,os.path.join(save_param_dir,'models.csv'))

### 確認: PC1のパラメータ抽出

In [3]:
%run ../tools/PC1.py
modelPC1 = PC1(gDF,rDF)

In [4]:
start = time.time()
pc1_fit = modelPC1.fit([-1.83033443, 2.74556604, 65.2612221, 74.74509713])
elapsed_time = time.time() - start
print('{} sec'.format(elapsed_time))

73.515914917 sec


In [5]:
ext = pc1_fit.extract()
type(ext), ext.keys()

(collections.OrderedDict, [u'ability', u'bias', u'noise', u'lp__'])

In [6]:
estimated = ext['ability'].mean(axis=0)
np.corrcoef(modelPC1.grade, estimated)[0,1]

0.23089013952775259

In [7]:
inferred_bias = ext['bias'].mean(axis=0)
inferred_noise = ext['noise'].mean(axis=0)

In [11]:
ext['noise'].mean(axis=0)

0.94108952004885216